In [18]:
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from activ import load_data, data_normalization
from sklearn.model_selection import train_test_split, cross_val_score
from activ.pipeline import score_clusters
import matplotlib.pyplot as plt
import scipy
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages
from pandas.tools.plotting import table

In [78]:
nmfdata = load_data(uoinmf=True)
oc_names = nmfdata.outcome_features

In [101]:
data = nmfdata
data_bases = data.outcome_bases
data_features = data.outcome_features
values = sorted(data_bases.sum(axis=0))[::-1]
ranked_features = data_features[np.argsort(-data_bases.sum(axis=0))]
values[0]

0.65971583873414386

In [106]:
np.argsort(-data_bases.sum(axis=0))[34]

89

In [107]:
data_features[89]

'RPQBlurredVision_6mo_R'

In [20]:
def rank_features(data):
    data_bases = data.outcome_bases
    data_features = data.outcome_features
    values = sorted(data_bases.sum(axis=0))[::-1]
    ranked_features = data_features[np.argsort(-data_bases.sum(axis=0))]
    return ranked_features, values

In [21]:
def drop_features(h,feature_names,percent):
    total_features = h.shape[1]
    num_features_to_drop = np.round(total_features*percent//100)
    h_sum = np.sum(h, axis=0)
    h_sum_sorted = np.argsort(h_sum)
    keep = h_sum_sorted[num_features_to_drop:]
    h_new = h[:,keep]
    fn_new = [feature_names[i] for i in keep][::-1]
    padding = ['']*num_features_to_drop
    return fn_new+padding

In [29]:
new10 = drop_features(nmfdata.outcome_bases, nmfdata.outcome_features, 10)
new20 = drop_features(nmfdata.outcome_bases, nmfdata.outcome_features, 20)
new30 = drop_features(nmfdata.outcome_bases, nmfdata.outcome_features, 30)
new40 = drop_features(nmfdata.outcome_bases, nmfdata.outcome_features, 40)
new50 = drop_features(nmfdata.outcome_bases, nmfdata.outcome_features, 50)
new60 = drop_features(nmfdata.outcome_bases, nmfdata.outcome_features, 60)
new70 = drop_features(nmfdata.outcome_bases, nmfdata.outcome_features, 70)
new80 = drop_features(nmfdata.outcome_bases, nmfdata.outcome_features, 80)
new90 = drop_features(nmfdata.outcome_bases, nmfdata.outcome_features, 90)


In [71]:
ranked, val = rank_features(nmfdata)
for x in range(len(ranked)):
    print(x, val[x], ranked[x])

0 0.659715838734 GOSE_OtherIssues6M
1 0.63440742189 NeuroSleepTroubleFallingAsleep6mo
2 0.626469938532 NeuroEmotionalIrritability6mo
3 0.622420656307 NeuroSleepSleepingLess6mo
4 0.548693129674 GOSE_Reponder6M_Patient.alone
5 0.542240115928 NeuroEmotionalSadness6mo
6 0.536659584763 GOSE_OtherIssues3M
7 0.530415045247 CHARTSFPhysicalTotal_6mo
8 0.520898032305 NeuroPhysVisualProbl6mo
9 0.512828260965 NeuroCognitiveDiffRemembering6mo
10 0.506431304282 NeuroEmotionalMoreEmotional6mo
11 0.49960774952 GOSE_Travel6M
12 0.498803349912 NeuroPhysFatigue6mo
13 0.492909013964 NeuroPhysBalanceProbl6mo
14 0.491638604044 CHARTSFSocialIntegrationTotal_6mo
15 0.488643653739 CHARTSFMobilityTotal_6mo
16 0.480085916849 CHARTSFInHomeAssistTime_6mo
17 0.477384608774 NeuroEmotionalNervousness6mo
18 0.476821920613 RPQSleepDisturbance_6mo
19 0.470284067463 CHARTSFCognitiveTotal_6mo
20 0.455596910479 RPQSleepDisturbance_6mo_R
21 0.44646959031 PCLAsleep_6mo
22 0.438989943745 NeuroCognitiveDiffRemembering3mo
23 0.

In [24]:
d = {'Leverage Score': val, '0%': ranked, '10%': new10, '20%':new20, '30%':new30, '40%':new40, 
     '50%': new50, '60%':new60, '70%':new70, '80%':new80, '90%':new90}
df = pd.DataFrame(data=d)
df.index+=1
df

,Leverage Score,0%,10%,20%,30%,40%,50%,60%,70%,80%,90%
1,0.659716,GOSE_OtherIssues6M,GOSE_OtherIssues6M,GOSE_OtherIssues6M,GOSE_OtherIssues6M,GOSE_OtherIssues6M,GOSE_OtherIssues6M,GOSE_OtherIssues6M,GOSE_OtherIssues6M,GOSE_OtherIssues6M,GOSE_OtherIssues6M
2,0.634407,NeuroSleepTroubleFallingAsleep6mo,NeuroSleepTroubleFallingAsleep6mo,NeuroSleepTroubleFallingAsleep6mo,NeuroSleepTroubleFallingAsleep6mo,NeuroSleepTroubleFallingAsleep6mo,NeuroSleepTroubleFallingAsleep6mo,NeuroSleepTroubleFallingAsleep6mo,NeuroSleepTroubleFallingAsleep6mo,NeuroSleepTroubleFallingAsleep6mo,NeuroSleepTroubleFallingAsleep6mo
3,0.626470,NeuroEmotionalIrritability6mo,NeuroEmotionalIrritability6mo,NeuroEmotionalIrritability6mo,NeuroEmotionalIrritability6mo,NeuroEmotionalIrritability6mo,NeuroEmotionalIrritability6mo,NeuroEmotionalIrritability6mo,NeuroEmotionalIrritability6mo,NeuroEmotionalIrritability6mo,NeuroEmotionalIrritability6mo
4,0.622421,NeuroSleepSleepingLess6mo,NeuroSleepSleepingLess6mo,NeuroSleepSleepingLess6mo,NeuroSleepSleepingLess6mo,NeuroSleepSleepingLess6mo,NeuroSleepSleepingLess6mo,NeuroSleepSleepingLess6mo,NeuroSleepSleepingLess6mo,NeuroSleepSleepingLess6mo,NeuroSleepSleepingLess6mo
5,0.548693,GOSE_Reponder6M_Patient.alone,GOSE_Reponder6M_Patient.alone,GOSE_Reponder6M_Patient.alone,GOSE_Reponder6M_Patient.alone,GOSE_Reponder6M_Patient.alone,GOSE_Reponder6M_Patient.alone,GOSE_Reponder6M_Patient.alone,GOSE_Reponder6M_Patient.alone,GOSE_Reponder6M_Patient.alone,GOSE_Reponder6M_Patient.alone
6,0.542240,NeuroEmotionalSadness6mo,NeuroEmotionalSadness6mo,NeuroEmotionalSadness6mo,NeuroEmotionalSadness6mo,NeuroEmotionalSadness6mo,NeuroEmotionalSadness6mo,NeuroEmotionalSadness6mo,NeuroEmotionalSadness6mo,NeuroEmotionalSadness6mo,NeuroEmotionalSadness6mo
7,0.536660,GOSE_OtherIssues3M,GOSE_OtherIssues3M,GOSE_OtherIssues3M,GOSE_OtherIssues3M,GOSE_OtherIssues3M,GOSE_OtherIssues3M,GOSE_OtherIssues3M,GOSE_OtherIssues3M,GOSE_OtherIssues3M,GOSE_OtherIssues3M
8,0.530415,CHARTSFPhysicalTotal_6mo,CHARTSFPhysicalTotal_6mo,CHARTSFPhysicalTotal_6mo,CHARTSFPhysicalTotal_6mo,CHARTSFPhysicalTotal_6mo,CHARTSFPhysicalTotal_6mo,CHARTSFPhysicalTotal_6mo,CHARTSFPhysicalTotal_6mo,CHARTSFPhysicalTotal_6mo,CHARTSFPhysicalTotal_6mo
9,0.520898,NeuroPhysVisualProbl6mo,NeuroPhysVisualProbl6mo,NeuroPhysVisualProbl6mo,NeuroPhysVisualProbl6mo,NeuroPhysVisualProbl6mo,NeuroPhysVisualProbl6mo,NeuroPhysVisualProbl6mo,NeuroPhysVisualProbl6mo,NeuroPhysVisualProbl6mo,NeuroPhysVisualProbl6mo
10,0.512828,NeuroCognitiveDiffRemembering6mo,NeuroCognitiveDiffRemembering6mo,NeuroCognitiveDiffRemembering6mo,NeuroCognitiveDiffRemembering6mo,NeuroCognitiveDiffRemembering6mo,NeuroCognitiveDiffRemembering6mo,NeuroCognitiveDiffRemembering6mo,NeuroCognitiveDiffRemembering6mo,NeuroCognitiveDiffRemembering6mo,NeuroCognitiveDiffRemembering6mo


In [27]:
df.to_csv('/Users/ahyeon/Desktop/activ misc/table.csv', sep=',')

In [72]:
for ii,v in enumerate(val):
    if ii>=36:
        val[ii] = ''

In [73]:
len(val)

177

In [74]:
d2 = {'Leverage Score': val[:36], '80%':new80[:36], '90%':new90[:36]}
df2 = pd.DataFrame(data=d2)
df2.index+=1
df2
df2.to_csv('/Users/ahyeon/Desktop/activ misc/smalltable.csv', sep=',')

In [76]:
d3 = {'Feature': new80[:36],'NMF Leverage Score': val[:36]}
df3 = pd.DataFrame(data=d3)
df3.index+=1
df3
df3.to_csv('/Users/ahyeon/Desktop/activ misc/finaltable.csv', sep=',')